In [47]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from time import mktime
import json
from geojson import Feature, FeatureCollection, Point

In [48]:
orginalPredictions = pd.read_csv('result_2022_07_20.csv', index_col=0)
orginalPredictions.head()

,day1,day1_confidence,day2,day2_confidence,day3,day3_confidence,day4,day4_confidence,day5,day5_confidence,latitude,longitude
0,0,-0.042379,0,-0.341115,1,0.557202,1,0.347046,0,-0.649949,49.006704,-122.727215
1,0,-0.080303,1,0.252081,1,0.262091,1,0.148855,0,-0.060581,49.006704,-122.714526
2,0,-0.080303,1,0.252081,1,0.262091,1,0.148855,0,-0.060581,49.006704,-122.676457
3,0,-0.080303,1,0.252081,1,0.262091,1,0.148855,0,-0.060581,49.006704,-122.625699
4,0,-0.080303,1,0.252081,1,0.262091,1,0.148855,0,-0.060581,49.006704,-122.613010


In [49]:
cur_date = date(2022, 7, 20)
delta = timedelta(days = 1)


In [50]:
days = []

for i in range(1, 6):
    curDay = (orginalPredictions[[f"day{i}", f"day{i}_confidence", "latitude", "longitude"]])
    curDay = curDay.dropna()
    cur_date_col = pd.DataFrame([cur_date]*len(curDay.index), columns=["date"])
    curDay = cur_date_col.join(curDay)
    cur_date += delta
    curDay = curDay.nlargest(500, f'day{i}_confidence')
    curDay.columns = pd.Series(['date', 'dayResult', 'dayConfidence', 'latitude', 'longitude'])
    days.append(curDay)


days

[              date  dayResult  dayConfidence   latitude   longitude
 89292   2022-07-20          1       0.735935  46.811421 -122.346530
 89293   2022-07-20          1       0.735935  46.811421 -122.333841
 89848   2022-07-20          1       0.735935  46.798731 -122.346530
 89849   2022-07-20          1       0.735935  46.798731 -122.333841
 89850   2022-07-20          1       0.735935  46.798731 -122.321151
 ...            ...        ...            ...        ...         ...
 106516  2022-07-20          1       0.735935  46.418046 -122.130809
 106517  2022-07-20          1       0.735935  46.418046 -122.118119
 106518  2022-07-20          1       0.735935  46.418046 -122.105430
 106519  2022-07-20          1       0.735935  46.418046 -122.092740
 106520  2022-07-20          1       0.735935  46.418046 -122.080051
 
 [500 rows x 5 columns],
              date  dayResult  dayConfidence   latitude   longitude
 89886  2022-07-21          1       0.540864  46.798731 -121.864329
 90441  2

In [51]:
allDays = pd.concat(days)
allDays = allDays[allDays["dayResult"] == 1]
allDays.insert(0, "SNo", range(0, len(allDays)))
allDays["dateUnixTimestamp"] = allDays.apply( lambda r: int(mktime(r["date"].timetuple())), axis=1 )
allDays["date"]
allDays

,SNo,date,dayResult,dayConfidence,latitude,longitude,dateUnixTimestamp
89292,0,2022-07-20,1,0.735935,46.811421,-122.346530,1658275200
89293,1,2022-07-20,1,0.735935,46.811421,-122.333841,1658275200
89848,2,2022-07-20,1,0.735935,46.798731,-122.346530,1658275200
89849,3,2022-07-20,1,0.735935,46.798731,-122.333841,1658275200
89850,4,2022-07-20,1,0.735935,46.798731,-122.321151,1658275200
...,...,...,...,...,...,...,...
10004,2495,2022-07-24,1,0.347503,48.727535,-117.626037,1658620800
10005,2496,2022-07-24,1,0.347503,48.727535,-117.613347,1658620800
10006,2497,2022-07-24,1,0.347503,48.727535,-117.600658,1658620800
10007,2498,2022-07-24,1,0.347503,48.727535,-117.587968,1658620800


In [52]:
features = allDays.apply(
    lambda row: Feature(properties={"id": row["SNo"], "mag": row["dayConfidence"], "time": row["dateUnixTimestamp"]}, geometry=Point((float(row["longitude"]), float(row["latitude"])))), axis=1
).tolist()

In [53]:
feature_collection = FeatureCollection(features=features, properties={"name": "single instance predictions"})

In [56]:
with open('/home/tejabalu/www/flaskapp/static/single_instance.geojson', 'w', encoding='utf-8') as f:
    f.write(json.dumps(feature_collection, ensure_ascii=False))


In [111]:
type(allDays["date"][0])

pandas.core.series.Series

In [113]:
import json
df = pd.DataFrame(
    [["a", "b"], ["c", "d"]],
    index=["row 1", "row 2"],
    columns=["col 1", "col 2"],
)
df

,col 1,col 2
row 1,a,b
row 2,c,d


In [116]:
result = df.to_json(orient="split")
parsed = json.loads(result)
json.dumps(parsed)  

'{"columns": ["col 1", "col 2"], "index": ["row 1", "row 2"], "data": [["a", "b"], ["c", "d"]]}'

In [117]:
result = df.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed)  

'{"row 1": {"col 1": "a", "col 2": "b"}, "row 2": {"col 1": "c", "col 2": "d"}}'

In [118]:
result = df.to_json(orient="columns")
parsed = json.loads(result)
json.dumps(parsed)  

'{"col 1": {"row 1": "a", "row 2": "c"}, "col 2": {"row 1": "b", "row 2": "d"}}'

In [130]:
df = pd.DataFrame([{'name': 'test', 'lat': 24.13604, 'lng': 120.6778}, {'name': 'test', 'lat': 24.13604, 'lng': 120.6778}, {'name': 'test', 'lat': 24.13604, 'lng': 120.6778}])
df

,name,lat,lng
0,test,24.13604,120.6778
1,test,24.13604,120.6778
2,test,24.13604,120.6778


In [131]:
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [147]:
# columns used for constructing geojson object
import time
print(type(cur_date))
curtimeunix = int(time.mktime(cur_date.timetuple())*1000)
print(curtimeunix)

features = df.apply( 
    lambda row: Feature(properties={"id": 1, "mag": 0.1, "time": curtimeunix}, geometry=Point((float(row['lng']), float(row['lat'])))), axis=1).tolist()

# all the other columns used as properties

# whole geojson object
feature_collection = FeatureCollection(features=features, properties={"name": "single instance predictions"})

<class 'datetime.date'>
1658707200000


In [148]:
json.dumps(feature_collection)

'{"type": "FeatureCollection", "properties": {"name": "single instance predictions"}, "features": [{"type": "Feature", "geometry": {"type": "Point", "coordinates": [120.6778, 24.13604]}, "properties": {"id": 1, "mag": 0.1, "time": 1658707200000}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [120.6778, 24.13604]}, "properties": {"id": 1, "mag": 0.1, "time": 1658707200000}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [120.6778, 24.13604]}, "properties": {"id": 1, "mag": 0.1, "time": 1658707200000}}]}'